In [1]:
import matplotlib.font_manager

system_fonts = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
indexes_to_remove = [17, 34, 44, 166, 174, 175, 178, 186, 263, 297, 317]
system_fonts = [font for i, font in enumerate(system_fonts) if i not in indexes_to_remove]

In [2]:
print(len(system_fonts))

324


In [3]:
from functions import (
    create_blank_pdf,
    find_optimal_placement,
    can_place_image,
    save_pdf, 
    place_image_in_pdf,
    create_text_image,
    get_all_image_bounding_boxes,
    test_image_placement,
    is_overlapping,
    append_image_to_pdf
)

In [ ]:
# Create text image
import random
import numpy as np
from reportlab.lib.pagesizes import letter

width, height = int(letter[0]), int(letter[1])

text = "This is sample text for our PDF"
words_per_line = np.random.binomial(n=10, p=0.5, size=1)[0]+3
pdf_path = "text_as_image.pdf"
font_path = random.choice(system_fonts)
font_size = np.random.binomial(n=25,p=0.5,size=1)[0]+1
text_image, text_info = create_text_image(text, words_per_line=words_per_line, font_path=font_path, font_size=font_size)

img, img_info = create_text_image(text)

pdf_path = "output.pdf"
canvas_obj, (page_width, page_height) = create_blank_pdf(pdf_path)
x_center, y_center = page_width/2, page_height/2
existing_boxes = []
if can_place_image(img, x_center, y_center, existing_boxes, page_width, page_height):
    place_image_in_pdf(canvas_obj, img, x_center, y_center)
    save_pdf(canvas_obj)
else:
    position = find_optimal_placement(img, existing_boxes, page_width, page_height)
    if position:
        x_center, y_center = position
        place_image_in_pdf(canvas_obj, img, x_center, y_center)
        save_pdf(canvas_obj)
    else:
        print("Cannot place image without overlaps")

In [ ]:


if __name__ == "__main__":
    text = "roses are red, violets are blue, and so are you"
    #x_centroid = random.randint(1, width)
    #y_centroid = random.randint(1, height)
    words_per_line = np.random.binomial(n=10, p=0.5, size=1)[0]+3
    pdf_path = "text_as_image.pdf"
    font_path = random.choice(system_fonts)
    font_size = np.random.binomial(n=25,p=0.5,size=1)[0]+1
    create_blank_pdf(pdf_path)
    text_image, text_info = create_text_image(text, words_per_line=words_per_line, font_path=font_path, font_size=font_size)
    x_centroid, y_centroid = find_optimal_placement(text_image, [], width, height)
    print(f"Created text image with dimensions: {text_info['image_width']}×{text_info['image_height']}")
    image_placement = append_image_to_pdf(pdf_path, text_image, x_centroid, y_centroid)
    print(f"Created PDF with text image positioned at centroid ({x_centroid}, {y_centroid})")
    bboxes = get_all_image_bounding_boxes(pdf_path)
    if bboxes:
        bbox = bboxes[0]
        print("\nImage bounding box in PDF:")
        print(f"Bottom-left: ({bbox['x0']:.2f}, {bbox['y0']:.2f})")
        print(f"Top-right: ({bbox['x1']:.2f}, {bbox['y1']:.2f})")
        print(f"Width × Height: {bbox['width']:.2f} × {bbox['height']:.2f}")
        print(f"Centroid: ({bbox['centroid_x']:.2f}, {bbox['centroid_y']:.2f})")
    else:
        print("Image not found in the PDF.")

C:\Windows\Fonts\ebrima.ttf
Created text image with dimensions: 207×66


IndexError: page 0 not in document

In [5]:
new_text = "Marroons are cool and GPT is fun"
max_retries = 15
retry_count = 0
new_bbox = None

while retry_count < max_retries:
    new_words_per_line = np.random.binomial(n=10, p=0.5, size=1)[0]+3
    new_font_path = random.choice(system_fonts)
    new_font_size = np.random.binomial(n=25,p=0.5,size=1)[0]+1
    new_x_centroid = random.randint(150, 450)
    new_y_centroid = random.randint(200, 600)
    new_text_image, new_text_info = create_text_image(new_text, words_per_line=new_words_per_line, font_path=new_font_path, font_size=new_font_size)
    new_bbox = test_image_placement(pdf_path, new_text_image, new_x_centroid, new_y_centroid)
    if not is_overlapping(bbox, new_bbox):
        append_image_to_pdf(pdf_path, new_text_image, new_x_centroid, new_y_centroid)
        print(f"Added new text successfully at ({new_x_centroid}, {new_y_centroid}) with font size {new_font_size}.")
        bbox = get_all_image_bounding_boxes(pdf_path)
        if bbox:
            print(bbox)
        break
    
    print(f"Overlap detected, retrying... ({retry_count + 1}/{max_retries})")
    retry_count += 1

if retry_count == max_retries:
    print("Failed to place new text without overlap after multiple attempts.")

TypeError: test_image_placement() takes 3 positional arguments but 4 were given